In [5]:
import requests

def register_face(name, image_path):
    try:
        with open(image_path, 'rb') as img:
            response = requests.post(
                "http://localhost:18081/face/register",
                files={"image": img},
                data={"name": name}
            )
            print(response.json())
    except requests.exceptions.ConnectionError as e:
        print(f"Could not connect to the server: {e}")

register_face("Amitabh", "Amitabh.jpg")
register_face("Utkarsh", "Utkarsh.jpg")


Could not connect to the server: HTTPConnectionPool(host='localhost', port=18081): Max retries exceeded with url: /face/register (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x1106b2060>: Failed to establish a new connection: [Errno 61] Connection refused'))
Could not connect to the server: HTTPConnectionPool(host='localhost', port=18081): Max retries exceeded with url: /face/register (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x108fe3650>: Failed to establish a new connection: [Errno 61] Connection refused'))


In [6]:
import cv2
import requests

video_path = "cctv_video.mp4"
cap = cv2.VideoCapture(video_path)
frame_num = 0
fps = int(cap.get(cv2.CAP_PROP_FPS))

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    if frame_num % fps == 0:  # one frame per second
        _, img_jpg = cv2.imencode('.jpg', frame)
        files = {'image': img_jpg.tobytes()}

        # Detect faces
        try:
            r = requests.post("http://localhost:18081/face/detect", files=files)
            detections = r.json().get('faces', [])
        except requests.exceptions.ConnectionError as e:
            print(f"Could not connect to the face detection server: {e}")
            break

        for face in detections:
            x1, y1, x2, y2 = map(int, face['bbox'])
            crop = frame[y1:y2, x1:x2]
            _, crop_jpg = cv2.imencode('.jpg', crop)
            try:
                match_r = requests.post("http://localhost:18081/face/search", files={"image": crop_jpg.tobytes()})
                match = match_r.json().get('results', [])[0]  # best match
                print(f"Frame {frame_num}: {match['name']} - Score: {match['score']:.2f}")
            except requests.exceptions.ConnectionError as e:
                print(f"Could not connect to the face search server: {e}")
                break

    frame_num += 1

cap.release()

Could not connect to the face detection server: HTTPConnectionPool(host='localhost', port=18081): Max retries exceeded with url: /face/detect (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x109036360>: Failed to establish a new connection: [Errno 61] Connection refused'))
